In [1]:
# Importing the libraries here

import pandas as pd
import numpy as np
import ast
import json
from operator import add 
from operator import truediv
import re
# import swifter
from tqdm import tqdm

In [2]:
#data with TECH removed or non master removed and tech removed
# summary = pd.read_csv("BERT_Pred_189_FULL_with_eng.csv")
# summary = pd.read_csv("BERT_Pred_189_FULL_without_MASTER.csv")
summary = pd.read_csv("BERT_Pred_500_without_MASTER_ROW_WISE.csv")
# summary = pd.read_csv("BERT_Pred_500_without_MASTER_ENG+TECH_ROW_WISE.csv")


# summary["BERT_tags"]=summary["BERT_tags_wo_MASTER"]
# summary["BERT_ENG_tag"]=summary["BERT_tags_wo_MASTER"]

In [3]:
summary.head()

,id,summaries,exact_matched_patt_contextual,Tech_from_string_match,BERT_Tags
0,d3sBpLsn19JAgs2I3eoQLw_0000,a multiskilled economist and project manager w...,['qlikview'],NaN,[]
1,d2xmOwryxUIws0gqCBzNMw_0000,a multiskilled economist and project manager ...,['qlikview'],"['able', 'and co', 'board', 'cat', 'cci', 'cin...",[]
2,bfP0BXlhcOgqCouGfIIbHw_0000,ive been involved in software development usi...,['openedge'],"['able', 'ace', 'ada', 'adapt', 'apt', 'flo', ...",[]
3,bdTTj0aGQRypq5vd-I1o-A_0000,information technology professional extensive...,['netsuite'],"['acl', 'ada', 'blis', 'cat', 'cien', 'ento', ...",[]
4,6pXOo5YPZzwFBbuhuBzjOQ_0000,experienced sales executive with a demonstrat...,['getswift'],"['ark', 'dot', 'driven', 'ets', 'getswift', 'g...",[]


In [4]:
dataa = summary[['id','summaries','exact_matched_patt_contextual','BERT_Tags']]

# dataa = summary[['ID','summaries','exact_matched_patt_contextual','string_matched']]

In [5]:
summary=dataa

In [6]:
summary = summary.sort_values(by=['id'], ascending=False)
summary.reset_index(inplace = True, drop = True)

In [7]:
col_name_op = "BERT_Tags"
# col_name_op = "string_matched"

In [8]:
# Pass the matched technology column that we got
def convert_data_format(df,column_name):
    
    for i in range(0, len(df)):
        
        if df[column_name][i][0] == '[':
            res = ast.literal_eval(df[column_name][i])
            df[column_name][i] = res
            
        else:
            x = []
            x.insert(0, df[column_name][i])
            df[column_name][i] = x
            
    return df

df_cm = summary

In [9]:
df_cm = convert_data_format(df_cm, col_name_op)

In [10]:
df_cm = convert_data_format(df_cm, "exact_matched_patt_contextual")

In [11]:
df_cm[col_name_op][12]

['azure']

In [12]:
df_cm["exact_matched_patt_contextual"][12]

['azure']

In [13]:
count_context_technologies = []

for i in range(0, len(df_cm)):
    count_context_technologies.extend(df_cm["exact_matched_patt_contextual"][i])
d = list(set(count_context_technologies))
len(d)

236

# Converting into dataformat to compare two strings

In [14]:
# TP: present in both sheet i==j
# TN: not in both (insignificant)
# FP: in extracted sheet but not in manual tagging     i!=j
# FN: in manual tagging and is not in extracted tagging j

In [15]:
tp = []
tn = []
fp = []
fn = []

In [16]:
TP = TN = FP = FN = 0

# TP + FP calculation
# op_string = exact_matched_patt
# , manual_tagged_string = Actual/Actual(NER)

df_cm.head()

,id,summaries,exact_matched_patt_contextual,BERT_Tags
0,zun1u5pEP6XGEgB8NnH5Cw_0000,after teaching at hogeschool rotterdam i foun...,[adobe],[]
1,zoxTMxOfZsWUN1PQnYNdWA_0000,i work and play at the intersection of busine...,[gmail],[]
2,zhYBF2y27LTSykBxobls2Q_0000,i am a clientfacing interior designer project ...,[crd],[]
3,zYOI4JmmTDiT023cU9g7XA_0000,having spent 8 months in 2011 working with dan...,[rsm],[]
4,zLkOjHFJxHyyLnDgBCBYgg_0000,years of helping clients both as a consultant...,[azure],[azure]


In [17]:
df_cm["exact_matched_patt_contextual"] = df_cm["exact_matched_patt_contextual"].fillna('[]')

# reading 
tech = pd.read_csv("/Users/mohitbagaria/Downloads/spacy_string_match/Eng_pydictionary_2.csv")
# tech = pd.read_csv("Tech_pydictionary_2.csv")

tech.columns = ["index", "title"]
tech.drop(columns = ["index"], axis =1 , inplace = True)
tech.reset_index(inplace = True, drop = True)

In [18]:
tech.head()

,title
0,Lattice
1,EMERGE Background Checks
2,Tenfold
3,Friday Feedback
4,Data Dwell


In [19]:
tech_list = tech["title"].tolist()
tech_list = list(set(tech_list))
len(tech_list)

5289

In [20]:
for i in range(0, len(tech_list)):
    tech_list[i] = tech_list[i].lower()

In [21]:
# tech_list

In [22]:
# df_cm['Eng_Label']=df_cm["exact_matched_patt_contextual"].copy()

In [23]:
tech_tag_list = []

def get_tech_tagged_data(manually_tagged, tech_list):
    keep=[]
    for word in manually_tagged:
        if word in tech_list:
            keep.append(word)
    return keep

In [24]:
# get_tech_tagged_data(df_cm["exact_matched_patt_contextual"][1],tech_list)

In [25]:
df_cm.head(10)

,id,summaries,exact_matched_patt_contextual,BERT_Tags
0,zun1u5pEP6XGEgB8NnH5Cw_0000,after teaching at hogeschool rotterdam i foun...,[adobe],[]
1,zoxTMxOfZsWUN1PQnYNdWA_0000,i work and play at the intersection of busine...,[gmail],[]
2,zhYBF2y27LTSykBxobls2Q_0000,i am a clientfacing interior designer project ...,[crd],[]
3,zYOI4JmmTDiT023cU9g7XA_0000,having spent 8 months in 2011 working with dan...,[rsm],[]
4,zLkOjHFJxHyyLnDgBCBYgg_0000,years of helping clients both as a consultant...,[azure],[azure]
5,zFG4JjhB6Rd6U7RzNXFBFA_0000,we are experts in lateral hiring for small mi...,[cascade],[]
6,yspXP4StzHCTTRBVpKP2PA_0000,clover patch australia provide highly competi...,[clover],[clover]
7,yez8nRSNcKDmp0Ho6PtK-A_0000,visionary bizdev leader with passion for crea...,[perl],[]
8,yPTGFhq7MTXI4RuVTZiggQ_0000,an experienced driven collaborative informati...,[dell],[dell]
9,yK5vFh3gxPOn1qbNXn-Wwg_0000,an accomplished it senior management professi...,[lob],[]


In [27]:
# df_cm['Tech_Label'] = df_cm.apply(lambda x: get_tech_tagged_data(x["MANUAL_TAG"], tech_list), axis = 1)
df_cm['Eng_Label'] = df_cm.apply(lambda x: get_tech_tagged_data(x["exact_matched_patt_contextual"], tech_list), axis = 1)

In [28]:
df_cm.head(10)

,id,summaries,exact_matched_patt_contextual,BERT_Tags,Eng_Label
0,zun1u5pEP6XGEgB8NnH5Cw_0000,after teaching at hogeschool rotterdam i foun...,[adobe],[],[adobe]
1,zoxTMxOfZsWUN1PQnYNdWA_0000,i work and play at the intersection of busine...,[gmail],[],[]
2,zhYBF2y27LTSykBxobls2Q_0000,i am a clientfacing interior designer project ...,[crd],[],[]
3,zYOI4JmmTDiT023cU9g7XA_0000,having spent 8 months in 2011 working with dan...,[rsm],[],[]
4,zLkOjHFJxHyyLnDgBCBYgg_0000,years of helping clients both as a consultant...,[azure],[azure],[azure]
5,zFG4JjhB6Rd6U7RzNXFBFA_0000,we are experts in lateral hiring for small mi...,[cascade],[],[cascade]
6,yspXP4StzHCTTRBVpKP2PA_0000,clover patch australia provide highly competi...,[clover],[clover],[clover]
7,yez8nRSNcKDmp0Ho6PtK-A_0000,visionary bizdev leader with passion for crea...,[perl],[],[]
8,yPTGFhq7MTXI4RuVTZiggQ_0000,an experienced driven collaborative informati...,[dell],[dell],[dell]
9,yK5vFh3gxPOn1qbNXn-Wwg_0000,an accomplished it senior management professi...,[lob],[],[lob]


In [ ]:
# df_cm.to_csv("technology_dataset_with_taggging.csv", index= False)
# df_cm.to_csv("non_technology_dataset_with_taggging.csv", index= False)

In [29]:
list_tech_dataset = []

In [30]:
# fill with 
for i in range(0, len(df_cm)):
    list_tech_dataset.extend(df_cm["Eng_Label"][i])

In [31]:
len(list_tech_dataset)

223

In [32]:
set_tech = set(list_tech_dataset)
len(set_tech)

101

In [33]:
for i in range(0, len(df_cm)):
    l = df_cm[col_name_op][i]
    df_cm[col_name_op][i] = [0 if element == '0' else element for element in l]

In [34]:
for i in range(0, len(df_cm)):
  words = df_cm[col_name_op][i]
  stopwords = [0]
  for word in list(words):
    if word in stopwords:
      words.remove(word)

In [35]:
def Diff(li1, li2): #FP entries
    return list(set(li2)-set(li1))

In [36]:
def Same(lis1,lis2): #TP entries
    return list(set(lis1).intersection(lis2))

In [37]:
def Common(lis1,lis2):
    return list(set(lis1)-(set(lis2)))

In [38]:
# list of technologies which are not in extracted list but are in tagged list i.e. FN entries
df_cm['FP_entries_eng'] = df_cm.apply(lambda x: Diff(x["Eng_Label"], x[col_name_op]), axis = 1)

In [39]:
# list of technologies which are in extracted list and are in tagged list i.e. TP entries
df_cm['TP_entries_eng'] = df_cm.apply(lambda x: Same(x["Eng_Label"], x[col_name_op]), axis = 1)

In [40]:
# list of technologies which are NOT extracted list and are in tagged list i.e. FN entries
df_cm['FN_entries_eng'] = df_cm.apply(lambda x: Common(x["Eng_Label"], x[col_name_op]), axis = 1)

In [41]:
df_cm.head(15)

,id,summaries,exact_matched_patt_contextual,BERT_Tags,Eng_Label,FP_entries_eng,TP_entries_eng,FN_entries_eng
0,zun1u5pEP6XGEgB8NnH5Cw_0000,after teaching at hogeschool rotterdam i foun...,[adobe],[],[adobe],[],[],[adobe]
1,zoxTMxOfZsWUN1PQnYNdWA_0000,i work and play at the intersection of busine...,[gmail],[],[],[],[],[]
2,zhYBF2y27LTSykBxobls2Q_0000,i am a clientfacing interior designer project ...,[crd],[],[],[],[],[]
3,zYOI4JmmTDiT023cU9g7XA_0000,having spent 8 months in 2011 working with dan...,[rsm],[],[],[],[],[]
4,zLkOjHFJxHyyLnDgBCBYgg_0000,years of helping clients both as a consultant...,[azure],[azure],[azure],[],[azure],[]
5,zFG4JjhB6Rd6U7RzNXFBFA_0000,we are experts in lateral hiring for small mi...,[cascade],[],[cascade],[],[],[cascade]
6,yspXP4StzHCTTRBVpKP2PA_0000,clover patch australia provide highly competi...,[clover],[clover],[clover],[],[clover],[]
7,yez8nRSNcKDmp0Ho6PtK-A_0000,visionary bizdev leader with passion for crea...,[perl],[],[],[],[],[]
8,yPTGFhq7MTXI4RuVTZiggQ_0000,an experienced driven collaborative informati...,[dell],[dell],[dell],[],[dell],[]
9,yK5vFh3gxPOn1qbNXn-Wwg_0000,an accomplished it senior management professi...,[lob],[],[lob],[],[],[lob]


In [42]:
# df_cm.to_csv("bert_eng_accuracy_M1_with_entries.csv", index = False)

In [43]:
def get_TP(df,manual_tagged_string,op_string):
    
    for ind in df.index:
        TP = FP = 0
        
        for i in range(0, len(df[op_string][ind])):
            
            if df[op_string][ind][i] in df[manual_tagged_string][ind]:
                TP = TP + 1
                
            else:
                pass
                
        tp.append(TP)

    return tp

In [44]:
fp_entry =[]
# fn_entry = []

In [45]:
# import numpy as np
# list_1 = ["a", "b", "c", "d", "e"]
# list_2 = ["a", "f", "c", "m"] 
# # yields the elements in `list_2` that are NOT in `list_1`
# main_list = np.setdiff1d(list_2,list_1)
# yields the elements in `list_2` that are NOT in `list_1`



def get_FP(df,manual_tagged_string, op_string):
    
    for ind in df.index:
        main_list = np.setdiff1d(df[op_string][ind],df[manual_tagged_string][ind])
        fp.append(len(main_list))
        fp_entry.append(main_list)
        
        
    return fp, fp_entry

In [46]:
# ORIGINAL
# def get_FN(df,manual_tagged_string, op_string):
    
#     for ind in df.index:
        
#         main_list = np.setdiff1d(df[manual_tagged_string][ind], df[op_string][ind])
#         fn.append(len(main_list))
        
#     return fn

In [47]:
def get_FN(df,manual_tagged_string, op_string):
    
    for ind in df.index:
        
        main_list = np.setdiff1d(df[manual_tagged_string][ind], df[op_string][ind])
        if df[manual_tagged_string][ind]=='[]':
            fn.append(0)
        else:
            fn.append(len(main_list))
        
    return fn

In [48]:
# entry like Empty|list of technology output


In [49]:
# def get_FN(df,manual_tagged_string, op_string):
    
#     for ind in df.index:
        
#         main_list = np.setdiff1d(df[manual_tagged_string][ind], df[op_string][ind])
#         if (manual_tagged_string[ind] == "[]"or manual_tagged_string[ind] == ""):
#             fn.append(0)
#         else:
#             fn.append(len(main_list))
        
#     return fn

In [50]:
# function call
tp = get_TP(df_cm, "Eng_Label", col_name_op)
fp, fp_entry_names = get_FP(df_cm, "Eng_Label", col_name_op)
fn = get_FN(df_cm, "Eng_Label", col_name_op)

# tp = get_TP(df_cm, "Tech_Label", col_name_op)
# fp, fp_entry_names = get_FP(df_cm, "Tech_Label", col_name_op)
# fn = get_FN(df_cm, "Tech_Label", col_name_op)


# tp = get_TP(df_cm, "Technology", col_name_op)
# fp = get_FP(df_cm, "Technology", col_name_op)
# fn = get_FN(df_cm, "Technology", col_name_op)

In [51]:
# tp

In [52]:
print(len(tp), len(fp), len(fn))

500 500 500


In [53]:
len(fp_entry_names)

500

In [54]:
type(fp_entry_names)

list

In [55]:
dict_fp = {}

In [56]:
for i in range(0, len(fp_entry_names)):
    fp_entry_names[i] = fp_entry_names[i].tolist()
    dict_fp[i] = fp_entry_names[i]

In [57]:
# dict_fp

In [58]:
# for i in range(0, len(fn_entry_names)):
# #     fn_entry_names[i] = fn_entry_names[i].tolist()
#     dict_fn[i] = fn_entry_names[i]

In [59]:
# dict_fn

In [60]:
# len(dict_fp)

In [61]:
# df_fp = pd.DataFrame(dict_fp.items(), columns=['index', 'FP_entries_string'])
# df_fn = pd.DataFrame(dict_fp.items(), columns=['index', 'FN_entries_combined'])

In [62]:
# len(df_fp)

In [63]:
# df_fp_fn = pd.concat([df_fp, df_cm], axis = 1)

In [64]:
# df_fp_fn.head()

In [65]:
# df_fp_fn.to_csv("FP_combined.csv", index = False)

In [66]:
# creating a dataframe
df_matrix = pd.DataFrame(list(zip(tp, fp, fn)), 
               columns =['TP', 'FP', "FN"])
df_matrix.head(-1)

,TP,FP,FN
0,0,0,1
1,0,0,0
2,0,0,0
3,0,0,0
4,1,0,0
5,0,0,1
6,1,0,0
7,0,0,0
8,1,0,0
9,0,0,1


In [67]:
def get_precision(tp,fp):
    
    if tp+fp ==0:
        precision = 0
    else:
        precision = tp/(tp+fp)
    
    return precision

In [68]:
def get_recall(tp,fn):
    
    if tp+fn ==0:
        recall = 0
    else:
        recall = tp/(tp+fn)
    
    return recall

In [69]:
def get_f_score(precision, recall):
    if precision+recall == 0:
        f_score = 0
    else:
        f_score = 2*(precision*recall)/(precision+recall)
        
    return f_score

In [70]:
df_matrix['Precision'] = df_matrix.apply(lambda x: get_precision(x["TP"], x["FP"]), axis = 1)

In [71]:
df_matrix["Recall"] = df_matrix.apply(lambda x: get_recall(x["TP"], x["FN"]), axis = 1)

In [72]:
df_matrix["f_score"] = df_matrix.apply(lambda x: get_f_score(x["Precision"], x["Recall"]), axis = 1)

In [73]:
df_matrix.head()

,TP,FP,FN,Precision,Recall,f_score
0,0,0,1,0.0,0.0,0.0
1,0,0,0,0.0,0.0,0.0
2,0,0,0,0.0,0.0,0.0
3,0,0,0,0.0,0.0,0.0
4,1,0,0,1.0,1.0,1.0


In [ ]:
# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_ner_en_large_flair_100_tagged.csv", index = False)
# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_spacy_adding_more_tags.csv", index = False)
# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_String_Match_non_context.csv", index = False)
# df_matrix.to_csv("189_BERT_tags_SPACY_TEST_DATA_M1_12_Jun_21.csv", index = False)

# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_lg__moreTAGS_tagged_100_more_ENTS.csv", index = False)
# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_lg_100_simple_spacy.csv", index = False)


In [74]:
final_data = df_cm.join(df_matrix)
final_data.to_csv("ACC-500_BERT_pred_on_ENG_ROW_WISE.csv")
# final_data.to_csv("1190_spacy_pred_tech_match.csv")